# Building a reputation score on the Blockchain for Bitcoin

### This is in intro that will describe our workflow process. Below you will find detailed steps on our thought process and how we are implemeting our project. 

##### Getting set up 
** Step 1: Open both this notebook and the BigQuery DB **

** Step 2: Import the Lib's & insert the BigQuery project ID **

##### Creating the DB using SQL & BigQuery 
** Step 3: Define the exchanges which will be used as a baseline for flagging **

** Step 4: Map each address and save it as a table to the BigQuery DB ** 

##### Setup middleware (decision tree) 
** Will take an input (address) and based on the flagging that is set up spit out a reputation score **


In [1]:
# Import Lib's 
import pandas as pd
from pandas.io import gbq
import random

In [2]:
# Insert the BigQuery Project ID
projectid = "capstone-202618"

### Define the exchanges

In [3]:
# Mt.Gox Hack 
mt_Gox_Hack = gbq.read_gbq('SELECT * FROM (FLATTEN(FLATTEN(DB.transactions_all, inputs.input_pubkey_base58_error), outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1LNWw6yCxkUmkhArb2Nf2MPw6vG7u5WG7q"', projectid)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=xFBfv9SyUGRfhe3ItekgCmFnX2ovM8&prompt=consent&access_type=offline
Enter the authorization code: 4/AACi44NWnaFeJGRCQE9RgUvmalMRV8XRTXTwDfiGLcnkGERFLtkxu8Q


### Creating transaction history per address
Group by public key (input), then build the transactions on that public key.
A Bitcoin address is an identifier of 26-35 alphanumeric characters that represents a possible destination for a bitcoin payment. 

In [3]:
# ## Get transaction table form BigQuery 
# transaction = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(DB.transactions_all, outputs.output_satoshis)) LIMIT 1250000', projectid)


In [16]:
transaction['inputs_input_pubkey_base58'].count()

159689

In [12]:
transaction = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(DB.transactions_all, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 IS NOT NULL LIMIT 50000', projectid)


In [17]:
for x in transaction['inputs_input_pubkey_base58']:
    try:
        pubKey = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(DB.transactions_all, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "{}"'.format(x), projectid)
        gbq.to_gbq(pubKey, 'DB.{}'.format(x), projectid, if_exists='replace')
    except KeyError, e:
        print("Error! type:{} - message:{}".format(str(type(e)), str(e)))


    print(x + " Mapped!!")
    


1N11ymJ5urLMXB3SxjoGzdupyaf3N7noDB Mapped!!
1N11ymJ5urLMXB3SxjoGzdupyaf3N7noDB Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1AcdF4HZdCJ1NLkhpe7wmi6tLPGyDAy32 Mapped!!
1BYCRXZfZqyZvkxS9eLoXTcomw8NufbbyY Mapped!!
1BYCRXZfZqyZvkxS9eLoXTcomw8NufbbyY Mapped!!
1NiUqRwuvkpSW7QaquiprSvPVWNXHTwgMJ Mapped!!
1NiUqRwuvkpSW7QaquiprSvPVWNXHTwgMJ Mapped!!
1NiUqRwuvkpSW7QaquiprSvPVWNXHTwgMJ Mapped!!
1EHS7ojM2xroewi8EMkaF1WknSy283yD9s Mapped!!
1FApr1RywsXvBWXckp7AHAUp9yr2YN5wXT Mapped!!
1PjVYStULFnwYKuXU9pbbTfmG8S82kqTNM Mapped!!
17ZSCZgCmnjotqTpwYW3bdugAkNxxT2hYb Mapped!!
17ZSCZgCmnjotqTpwYW3bdugAkNxxT2hYb Mapped!!
1MRzfGLe4eV4a3Dmg3zccMSB68ozMChKeG Mapped!!
16zBdpGo9PxKU7sxs7Pot676jxeZskRB2Y Mapped!!
14zH9AZaXZ4rDcVubWYNGjj2jnYnQNseZL Mapped!!
1EHS7ojM2xroewi8EMkaF1WknSy283yD9s Mapped!!
1FApr1RywsXvBWXckp7AHAUp9yr2YN5wXT Map

SSLError: HTTPSConnectionPool(host='www.googleapis.com', port=443): Max retries exceeded with url: /bigquery/v2/projects/capstone-202618/jobs (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [18]:
address17 = gbq.read_gbq('SELECT timestamp, transaction_id, inputs.input_pubkey_base58, outputs.output_satoshis, outputs.output_pubkey_base58 FROM (FLATTEN(DB.transactions_all, outputs.output_satoshis)) WHERE inputs.input_pubkey_base58 == "1FoxBitjXcBeZUS4eDzPZ7b124q3N7QJK7"', projectid)
# gbq.to_gbq(address17, 'DB.12sj9d85J3NwWTYuE9oPnAq5YyDhsrmnVn', projectid)
address17['inputs_input_pubkey_base58'].count()

560168

In [9]:
## First select a random public key from the data 
def randomPublic(df, n):
    return df.loc[random.sample(df.index, n)]

In [14]:
randomPublic(transaction, 15)

,timestamp,transaction_id,inputs_input_pubkey_base58,outputs_output_satoshis,outputs_output_pubkey_base58
279753,1507219686000,2c9a99de26468b0e5c451ed9649460073d31e3846a0ee2...,1JycrXp9cvpRyQ3emYHT26CfztDa6hfwGn,5000000000,12oLTdrRshL8ELVZ8rHUWKWpYGqYCiHC7a
1596060,1507050772000,5998f059f74f763801fe5d9f83690e5fc8dbbd7a8e1838...,None,9689186,185AmYcWwK8UTCR3MV8ttiWD92zHLeopte
1267711,1516749380000,9dff1f06ef123bdf745bf9e82e3a7b9187fe4c45fcd4fa...,19DWqGXPag2tKck7NaYEF3Waw6Uu52qX7F,252796,1PuaX6T93xvb7HKCUnHPVd8trrP5yDQB3E
414868,1415535752000,ea16a5de97740704aae9eb760d57f18cabfc215afb39d9...,1CiMNvP31FMyhAvf52QC4S1BpB64i5Pm4F,500000,1E4BwJweYHXgUP624AxHjiNZs5yA5wuCox
1689473,1515446008000,d93e41f7d2d350d57b2401bf6dfab69d73c82a84a3f841...,16gr6hH6nTjq2x7fk7eYGSTMzv3y5Suntp,10638410,12pPyf3mqb3pfnWvuhkVNFkYEJchuGjMqL
809202,1459490846000,7c5437b7180993cc3a49d3c64544e90c9c3f8c0d719629...,1K1QucZvABaF15ZxSbYMGD3pwuCBC6zuJE,2169021279,1BRebmXwXB35NyXXiJCvG2mUhogq6ftoLV
1777830,1436974064000,5683143007027c9192947b1891213bc51c35f36bf03e29...,1Edy1riJGUmewWxkQJjZyK1YRLmWPZ3tLC,2480000000,1Px7EcikQ5pgoM2duexuHASBaGwvYfbJVs
1087376,1412950144000,fa43e3929436e30463f0a35336ecdd41f8679ddd3fd01a...,18YR6VPCgRtr5bTYNo1BkG9fiHCun3ndAn,1493853,1FAVbTRU5NWBXbtoSRJZ4z3PJcG5gw6Bfs
274544,1448768758000,9b25c292249b389d53766fd34001d6c4d9fc768ba8db30...,1ECGMNDeFgG9Xh5iNkzzLxzrmqtHHm2C9P,400126874,16z15JZDT2ewJHpwAYUh9aVBAS5G926P2m
320069,1479504985000,32af2b176c89b3e55c2a0f6ad019ce73afb9065909d0e6...,12WpZXSdkp4kXkEK2ttZegRebaFsunex9t,79276983,1eGgnvifEeVAMK3Wv82kxbny9rizX6VR9


In [1]:
## Start creating the tranaction history per address
#  map the adresses (Pay-to-PubkeyHash)
#  then build the tranactions on that public key

## pub with max at the top (rank)
def pubKey(df, x):
    return df.loc[lambda df: df.inputs_input_pubkey_base58 == x, :]

In [2]:
pubKey(transaction, '1FoxBitjXcBeZUS4eDzPZ7b124q3N7QJK7')

NameError: name 'transaction' is not defined